In [1]:
# Here we gonna build 2 Models

# First for User like the product or not ("Yes" or "No")
# Second for Find review or rating for the product

### Importing the Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import variable

### Importing the dataset

In [3]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')
# ratings contains 1 Million data, so it take some time
print(movies, users, ratings)

         0                                   1                             2
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
...    ...                                 ...                           ...
3878  3948             Meet the Parents (2000)                        Comedy
3879  3949          Requiem for a Dream (2000)                         Drama
3880  3950                    Tigerland (2000)                         Drama
3881  3951             Two Family House (2000)                         Drama
3882  3952               Contender, The (2000)                Drama|Thriller

[3883 rows x 3 columns]          0  1   2   3      4
0        1  F   1  10 

 ### Preparing the Training set and test set

In [4]:
# The users movies rating in the ml 100k folder
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')

# Converting the csv file to array
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

### Gettting the number of users and movies

In [5]:
nb_users = int(max(max(training_set[:,0]), max(training_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(training_set[:,1])))
print(nb_users)
print(nb_movies)


943
1682


### Converting the data into an array with users in lines and movies in columns

In [6]:
# function to convert data into an required format

# it is a list of lists
# First list is 943 users, inside the each user it have another list
# The second list have rating of all movies given by that particular user

def convert(data):
    new_data = []   # This is the final list of lists
    
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]    # We can mention the condition like this
        id_ratings = data[:, 2][data[:, 0] == id_users]

        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings

        new_data.append(list(ratings))
        
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)
# print(training_set)
# print(test_set)

### Converting the data into Torch tensors

In [7]:
# The tensor converts all element into a single data type ( Here we convert into FLOAT )

# The floattensor requires List of Lists only
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

### Convertinig the ratings into binary ratings 1 (Liked) or 0 (Not liked)

In [8]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Here we just convert into os and 1s, if not rated converted to -1

### Creating the architecture of the Neural Network

In [21]:
class RBM():
    def __init__(self, nv, nh):   # num of visible and  hidden nodes
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh) # Bias for hidden nodes
        self.b = torch.randn(1, nv) # Bias for visible nodes

    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
        
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
        
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0).t() - torch.mm(vk.t(), phk).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

### Training the RBM

In [25]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.2448)
epoch: 2 loss: tensor(0.2487)
epoch: 3 loss: tensor(0.2461)
epoch: 4 loss: tensor(0.2497)
epoch: 5 loss: tensor(0.2459)
epoch: 6 loss: tensor(0.2476)
epoch: 7 loss: tensor(0.2444)
epoch: 8 loss: tensor(0.2488)
epoch: 9 loss: tensor(0.2454)
epoch: 10 loss: tensor(0.2488)


### Testing the RBM

In [26]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2322)
